In [9]:
today_date = ''

StatementMeta(, ec6a3fb6-5bcb-42e3-bcb7-d9ac35fe3b88, 11, Finished, Available, Finished)

In [4]:
abfs_path = 'abfss://NycTaxiProject@onelake.dfs.fabric.microsoft.com/NycTaxiProject.Lakehouse/Files/Landing/NycTaxigreen'
partition_path = f"/processing_date={today_date}"
complete_path = abfs_path + partition_path
print(complete_path)



StatementMeta(, ec6a3fb6-5bcb-42e3-bcb7-d9ac35fe3b88, 6, Finished, Available, Finished)

abfss://NycTaxiProject@onelake.dfs.fabric.microsoft.com/NycTaxiProject.Lakehouse/Files/Landing/NycTaxigreen/processing_date=2026-01-13


In [5]:
from pyspark.sql.functions import col, to_date, col, trim, regexp_replace, when, expr, to_timestamp, date_format, lit, isnull, hour, concat_ws, sha2, dayofweek

nyctaxigreen_landing_path = "abfss://NycTaxiProject@onelake.dfs.fabric.microsoft.com/NycTaxiProject.Lakehouse/Files/Landing/NycTaxigreen"

df_green = spark.read.format('csv').option("header", "true").load(nyctaxigreen_landing_path).filter(col('processing_date')==str(today_date))

StatementMeta(, ec6a3fb6-5bcb-42e3-bcb7-d9ac35fe3b88, 7, Finished, Available, Finished)

In [6]:
%%sql
CREATE TABLE IF NOT EXISTS nyctaxigreen_bronze (
    VendorID string,
    tpep_pickup_datetime string,
    tpep_dropoff_datetime string,
    passenger_count string,
    trip_distance string,
    RatecodeID string,
    store_and_fwd_flag string,
    PULocationID string,
    DOLocationID string,
    payment_type string,
    fare_amount string,
    extra string,
    mta_tax string,
    tip_amount string,
    tolls_amount string,
    improvement_surcharge string,
    total_amount string,
    congestion_surcharge string,
    Airport_fee string,
    cbd_congestion_fee string,
    processing_date date
)

USING DELTA
PARTITIONED BY (processing_date);

StatementMeta(, ec6a3fb6-5bcb-42e3-bcb7-d9ac35fe3b88, 8, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [7]:
from pyspark.sql.functions import col, lit
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

(
    df_green
    .withColumn("processing_date", lit(today_date))
    .write
    .format("csv")
    .mode("overwrite")
    .option("header", "true")
    .partitionBy("processing_date")
    .saveAsTable("nyctaxigreen_bronze")
)

StatementMeta(, ec6a3fb6-5bcb-42e3-bcb7-d9ac35fe3b88, 9, Finished, Available, Finished)

In [8]:
df = spark.sql("SELECT * FROM NycTaxiProject.nyctaxigreen_bronze")
df.count()

StatementMeta(, ec6a3fb6-5bcb-42e3-bcb7-d9ac35fe3b88, 10, Finished, Available, Finished)

55399